In [412]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import pandas as pd
import numpy as np

In [663]:
for pkm_no in range(19, 802):
    html = open('/Users/mac/Desktop/projects/pokemon/pokedex/{}.html'.format(pkm_no)).read()
    soup = BeautifulSoup(html, 'html.parser')
    data_cleaning(pkm_no, soup)
    

In [662]:
def data_cleaning(pkm_no, soup):
    def ability_extract(tag='a', rex='特性', num=1):
        tbody = soup.find_all(tag, text=re.compile(rex), attrs={})[num].parent.parent.parent
        tbody_row = tbody.find_all('tr')
        ability = (rex, tbody_row[1].find('td').text)
        ability2 = (None)
        try:
            ability2 = (rex,tbody_row[3].find('a').text)
            return ability, ability2
        except:
            pass
        return ability

    def add_number(ability_list):
        return_list = [(y[0]+str(x+1), y[1]) for x,y in enumerate(ability_list)]
        return return_list
    
    try:
        normal_ability = add_number([ability_extract(tag='a', rex='特性', num=1)])
    except:
        normal_ability = add_number(ability_extract(tag='a', rex='特性', num=1))
    try:
        special_ability = add_number([ability_extract(tag='a', rex='夢特', num=0)])
    except:
        special_ability = add_number(ability_extract(tag='a', rex='夢特', num=0))
    """---------------------------"""
    # 進化
    def evolution_extract():
        try:
            tbody = soup.find('td', text=re.compile('進化'), attrs={}).parent.parent
        except:
            tbody = soup.find('th', text=re.compile('進化'), attrs={}).parent.parent
        list_evolution_raw = [x.text for x in tbody.find_all('td')]
        try:    
            list_evolution = [list_evolution_raw[slice(1+8*x,9+8*x)] for x in range(0,2)]
        except:
            list_evolution = list_evolution_raw[slice(1,9)]
        return list_evolution

    evolution_list = evolution_extract()
    evolution_list = [''.join(x) for x in evolution_list] 
    evolution_list = [('進化'+str(x), y) for x, y in enumerate(evolution_list)]


    """---------------------------"""

    # 轉換種族值的方法
    strength = soup.find('td', text=re.compile('種族值')) # 在html 中的location
    strength_table = strength.parent.find_all('td')
    hp = strength_table[1]
    atk = strength_table[2]
    df = strength_table[3]
    spAtk = strength_table[4]
    spDf = strength_table[5]
    spd = strength_table[6]
    strength_text = [hp, atk, df, spAtk, spDf, spd]

    k =['hp', 'atk', 'df', 'spAtk', 'spDf', 'spd']
    strength_int = list(map(lambda x: int(x.text), strength_text))
    strength_tuple = list(zip(k, strength_int))# 用來整一個dictionary

    """---------------------------"""

    # 全國編號, 地區編號, 日文名稱, 中文名稱, 英文名稱, 屬性, 身高, 體重, 蛋組, 孵化步數, 性別比例, 總經驗值, 初始親密, 捕捉機率
    def extract_row_data(x):
        return x.find_all('td')[0].text, x.find_all('td')[1].text

    def basic_data_extract(tag='td', name='全國編號', num=0):
        tbody  = soup.find_all(tag, text=re.compile(name), attrs={})[num].parent.parent # td -> tr -> tbody
        tbody_row = tbody.find_all('tr')
        return list(map(extract_row_data, tbody_row))

    # 將基本資料、進化和特性合併
    basic_data_list = basic_data_extract()
    basic_data_list+=evolution_list
    basic_data_list+=normal_ability
    basic_data_list+=special_ability
    basic_data_list+=strength_tuple

    """---------------------------"""

    # 找招式相關的地方
    def skill_extract(th_name):
        th_position = soup.find('a', text=re.compile(th_name), attrs={})
        skill_tbody = th_position.parent.parent.parent     # special_skill 的 tbody
        skill_list = skill_tbody.find_all('tr')
        extracted_skill_list = []
        for x in skill_list[1:]:
            skill_row = x.find_all('td')
            requirement = skill_row[0].text
            try:
                cn_name = skill_row[1].text
            except:
                cn_name = None
                
            
            extracted_skill_list.append((requirement, cn_name))
            try:
                requirement = skill_row[2].text
                cn_name = skill_row[3].text
                extracted_skill_list.append((requirement, cn_name))
            except:
                pass
        return extracted_skill_list

    lv_skill_list = skill_extract('升級技能')
    try:
        special_skill_list = skill_extract('遺傳技能')
    except:
        special_skill_list = (0,0)
    HM_skill_list = skill_extract('日月機器技能')

    """-------------# 處理 lv_skill_list, special_skill_list, HM_skill_list 的合併-------------"""

    special_skill_df = pd.DataFrame(special_skill_list, columns= ['招式', '學習條件'])
    HM_skill_df = pd.DataFrame(HM_skill_list, columns=['學習條件','招式'])
    lv_skill_df = pd.DataFrame(lv_skill_list, columns=['學習條件','招式'])
    skills_df = pd.concat([lv_skill_df, HM_skill_df, special_skill_df], sort=False)
    skills_df.to_csv('/Users/mac/Desktop/projects/pokemon/pokedex/pkm_skill/skills-{}.csv'.format(pkm_no), index=False)

    """-------------出︰基本資料 和 招數-------------"""
    basic_data_df = pd.DataFrame(basic_data_list) # 將基本資料變做 df
    basic_data_df = basic_data_df.T
    basic_data_df.columns=basic_data_df.iloc[0] # 將第一行變成了做 columns
    basic_data_df.drop(0, axis=0, inplace=True)
    basic_data_df.index = [0]
    basic_data_df.to_csv('/Users/mac/Desktop/projects/pokemon/pokedex/pkm_skill/basic_data-{}.csv'.format(pkm_no), index=False)


In [595]:
data_cleaning(10, soup)


IndexError: list index out of range

In [596]:
def basic_data_extract(tag='td', name='全國編號', num=0):
    tbody  = soup.find_all(tag, text=re.compile(name), attrs={})[num].parent.parent # td -> tr -> tbody
    tbody_row = tbody.find_all('tr')
    return list(map(extract_row_data, tbody_row))

    
basic_data_extract()
    basic_data_list+=evolution_list
    basic_data_list+=normal_ability
    basic_data_list+=special_ability
    basic_data_list+=strength_tuple


[('全國編號', '010'),
 ('阿羅拉', '017'),
 ('日文名稱', 'キャタピー'),
 ('中文名稱', '綠毛蟲'),
 ('英文名稱', 'Caterpie'),
 ('屬性', '蟲'),
 ('身高', '0.3m / 1\'00"'),
 ('體重', '2.9kg / 6.4lbs'),
 ('蛋組', '昆蟲類'),
 ('孵化步數', '3840'),
 ('性別比例', '♂ 1 : ♀ 1'),
 ('總經驗值', '1000000'),
 ('初始親密', '70'),
 ('捕捉機率', '255')]

In [598]:
def skill_extract(th_name):
    th_position = soup.find('a', text=re.compile(th_name), attrs={})
    skill_tbody = th_position.parent.parent.parent     # special_skill 的 tbody
    skill_list = skill_tbody.find_all('tr')
#     extracted_skill_list = []
#     for x in skill_list[1:]:
#         skill_row = x.find_all('td')
#         requirement = skill_row[0].text
#         cn_name = skill_row[1].text
#         extracted_skill_list.append((requirement, cn_name))
#         try:
#             requirement = skill_row[2].text
#             cn_name = skill_row[3].text
#             extracted_skill_list.append((requirement, cn_name))
#         except:
#             pass
#     return extracted_skill_list

lv_skill_list = skill_extract('升級技能')
try:
    special_skill_list = skill_extract('遺傳技能')
except:
    special_skill_list = (0,0)
HM_skill_list = skill_extract('日月機器技能')


In [599]:
lv_skill_list

In [603]:
th_position = soup.find('a', text=re.compile('遺傳技能'), attrs={})
skill_tbody = th_position.parent.parent.parent     # special_skill 的 tbody
skill_list = skill_tbody.find_all('tr')

In [604]:
skill_tbody

<tbody><tr><th colspan="2"><a title="共0招">遺傳技能</a> / <a title="共0招">特別技能</a></th></tr>
<tr><td class="center" colspan="2">沒有相關技能</td></tr></tbody>

In [609]:
a=None
k = a if a else 'see'

In [610]:
k

'see'

In [630]:
def ability_extract(tag='a', rex='特性', num=1):
    tbody = soup.find_all(tag, text=re.compile(rex), attrs={})[num].parent.parent.parent
    tbody_row = tbody.find_all('tr')
    ability = (rex, tbody_row[1].find('td').text)
    ability2 = (None)
    try:
        ability2 = (rex,tbody_row[3].find('a').text)
        return ability, ability2
    except:
        pass
    return ability

def add_number(ability_list):
    return_list = [(y[0]+str(x+1), y[1]) for x,y in enumerate(ability_list)]
    return return_list


ability_list = ability_extract(tag='a', rex='特性', num=1)
ability_list
return_list = [(y[0]+str(x+1), y[1]) for x,y in enumerate(ability_list)]
return_list

[('特性1', '逃跑'), ('特性2', '毅力')]

In [650]:
def ability_extract(tag='a', rex='特性', num=1):
        tbody = soup.find_all(tag, text=re.compile(rex), attrs={})[num].parent.parent.parent
        tbody_row = tbody.find_all('tr')
        ability = (rex, tbody_row[1].find('td').text)
        ability2 = (None)
        try:
            ability2 = (rex,tbody_row[3].find('a').text)
            return ability, ability2
        except:
            pass
        return ability

def add_number(ability_list):
    return_list = [(y[0]+str(x+1), y[1]) for x,y in enumerate(ability_list)]
    return return_list

normal_ability = ability_extract(tag='a', rex='特性', num=1)
return_list = [(x,y[0]) for x,y in enumerate(normal_ability)]
normal_ability

[(0, '特性'), (1, '特性')]

In [655]:
def evolution_extract():
    tbody = soup.find('td', text=re.compile('進化'), attrs={}).parent.parent
    list_evolution_raw = [x.text for x in tbody.find_all('td')]
    try:    
        list_evolution = [list_evolution_raw[slice(1+8*x,9+8*x)] for x in range(0,2)]
    except:
        list_evolution = list_evolution_raw[slice(1,9)]
    return list_evolution

evolution_list = evolution_extract()
evolution_list = [''.join(x) for x in evolution_list] 
evolution_list = [('進化'+str(x), y) for x, y in enumerate(evolution_list)]

AttributeError: 'NoneType' object has no attribute 'parent'

In [661]:
tbody = soup.find('th', text=re.compile('進化'), attrs={})
tbody

<th colspan="7">進化</th>